# Divide And Conquer


This is part 1 of my notes for the [algorithms illuminated](https://youtube.com/playlist?list=PLEGCF-WLh2RLHqXx6-GZr_w7LgqKDXxN_&si=F_Br-YQVAlhE8GxM) youtube series.

The divide and conquer algorithm paradigm involves the following three steps:
1. Break the problem into different subproblems
2. Conquer each of the subproblem separately
3. Merge the results of all the subproblems to solve the bigger problem (this is where most of the ingenuity lies).


## Merge Sort Algorithm

The merge sort algorithm it the canonical Divide and Conquer example. 
It improves over traditional sorting algorithms as it is $O(nlog(n)$ while other sorting algorithms are typically $O(n^2)$.


**Input**: An unsorted list of integers

**Output**: A sorted list of integers.


In [2]:
fn merge_sort(input: Vec<i8>) -> Vec<i8> {
    if (input.len() <= 1) {
        return input;
    }
    // Split the input into 2 halfs
    let mid_index = input.len() / 2;
    let vec_a = (&input[0..mid_index]).to_vec();
    let vec_b = (&input[mid_index..input.len()]).to_vec();

    // Sort the 2 halfs
    let vec_a = merge_sort(vec_a);
    let vec_b = merge_sort(vec_b);
    
    // Merge the 2 halfs
    let mut output: Vec<i8> = Vec::new();
    let mut i = 0;
    let mut j = 0;
    for k in 0..input.len() {
        if j >= vec_b.len() || (i < vec_a.len() && vec_a[i] <= vec_b[j]) {
            output.push(vec_a[i]);
            i += 1;
        }
        else {
            output.push(vec_b[j]);
            j += 1;
        }
    }
    return output;
}

println!("output {:?}", merge_sort(vec![10, 20, 5]));

output [5, 10, 20]


In [3]:
// Let's now test the merge_sort algorithm
fn test_merge_sort(function: fn(Vec<i8>) -> Vec<i8>) {
    // Empty vector
    assert_eq!(function(vec![]), vec![]);
    // Vector of length 1
    assert_eq!(function(vec![10]), vec![10]);
    // Odd vector length
    assert_eq!(function(vec![2, 3, 1]), vec![1, 2, 3]);
    // Even vector length
    assert_eq!(function(vec![2, 3, 1, 4]), vec![1, 2, 3, 4]);
    // Large vector
    assert_eq!(function(vec![-1, 2, 30, 40, -19, 30]), vec![-19, -1, 2, 30, 30, 40]);
}

test_merge_sort(merge_sort);

Let's now analyze the time and space complexity of this algorithm.

### Time complexity

Since the algorithm is recursive, let's look at the recursion tree:

![recursion_tree](../diagrams/exported/recursion_tree.png)


- We have $log_2(n)$ levels of the tree.
- The merge operation is linear in the size of the input.
- We have $2^{level}$ nodes at every level, each node has an input of size $\frac{n}{2^{level}}$
- Hence at every level the amount of work done is = work_per_node x num_nodes = $\frac{n}{2^{level}}$ x $2^{level}$ = n
- The overall complexity is = work_per_level * num_levels = $n log_2(n)$


## Count the Number of inversions

This is another classic divide an conquer algorithm.

This is a problem where we want to count the number of inversions in an unsorted array. For an array $A$, an inversion is defined as $A[j] \gt A[i]$ where $j \gt i$.

**Input**: An unsorted array of length $n$.

**Output**: An integer representing the number of inversions in the array.

In [4]:
// Let's first define the test cases

fn test_count_inversions(function: fn(&mut [i8]) -> usize) {
    // Sorted array should have zero inversions.
    assert_eq!(function(&mut vec![1, 2, 3, 4]), 0);
    // Sorted array in desecnding order should have n(n-1)/2 inversions
    assert_eq!(function(&mut vec![4, 3, 2, 1]), 6);
    // Random array
    assert_eq!(function(&mut vec![1, 3, 6, 2, 5]), 3);
}

In [5]:
// Let's now implement the count inversions method
// We will piggy back over the merge sort algorithm

fn mergesort_countinversions(input: &mut [i8]) -> usize {
    
    // Base case
    if (input.len() <= 1) {
        return 0;
    }
    
    let mid_index = input.len() / 2;
    
    let num_inversions_left = mergesort_countinversions(&mut input[..mid_index]);
    let num_inversions_right = mergesort_countinversions(&mut input[mid_index..]);
    
    let mut num_inversions_split: usize = 0;
    let mut i = 0;
    let mut j = mid_index;
    
    let mut sorted_output: Vec<i8> = Vec::new();
    
    
    while (i < mid_index || j < input.len()) {
        if (j >= input.len() || (i < mid_index && input[i] <= input[j])) {
            sorted_output.push(input[i]);
            i += 1;
        }
        else {
            sorted_output.push(input[j]);
            j += 1;
            num_inversions_split += (mid_index - i);
        }
    }
    input.copy_from_slice(&sorted_output);
    return num_inversions_left + num_inversions_right + num_inversions_split;
}
test_count_inversions(mergesort_countinversions);

So let's now analyze the time and space complexity of the above algorithm:
Same as merge sort, the time complexity is $nlog(n)$.
As for the space complexity, it is O(n) mainly because of the copy `sorted_output` vector that we have to create.

## Matrix Multiplication
The traditional matrix multiplication algorithm involves the following.
- Assuming you have A and B matrices that are both n x n, multiplying them out results in an output matrix C which is also n x n.
- The traditional matrix multiplication method involves taking the dot product of the ith row from matrix A against the jth column of matrix B. Illustrated below:

$$
C_{ij} = A_{i_th\ row} \cdot B_{jth\ column}
$$

- This algorithm is $O(n^3)$, since:
   1. C has $n^2$ values
   2. For each value of C, the dot product is $O(n)$
   Hence, the overall complexity of the traditional matrix multiplication is $O(n^3)$


### Divide and Conquer in matrix multiplication

One might think that the divide and conquer paradigm may work with matrix multiplication to improve the performance over the traditional method. However, it doesn't work, here is why:

Assuming that we will divide each of the matrices A,B and C into there respective quadrants as such:

$$
\begin{bmatrix}
A1 & A2 \\
A3 & A4 \\
\end{bmatrix}
\begin{bmatrix}
B1 & B2 \\
B3 & B4 \\
\end{bmatrix} =
\begin{bmatrix}
A1 \cdot B1 + A2 \cdot B3 & A1 \cdot B2 + A2 \cdot B4 \\
A3 \cdot B1 + A4 \cdot B3 & A3 \cdot B2 + A4 \cdot B4 \\
\end{bmatrix}
$$

We could then perform 8 recursive calls to compute the subproducts:
1. $A1 \cdot B1$
2. $A1 \cdot B2$
3. $A2 \cdot B3$
4. $A2 \cdot B4$
5. $A3 \cdot B1$
6. $A3 \cdot B2$
7. $A4 \cdot B3$
8. $A4 \cdot B4$

Then merge the result adding the appropriate subcomponents together to form the final matrix C.

The complexity of this algorithm is still $O(n^3)$ but why?


Let's again create the recursive tree to show why:

![mm_recursion](../diagrams/exported/mm_recursion.png)


- The divide / conquer step: We have $log_2(n)$ levels, each level has $8^{level}$ nodes.
- The merge step: For each level, we have to peforms 4 additions, each addition is $(n / 2^{level})^2$. That's because for each level, we have to add all the 8 products together to form the output. Adding a single product is $(n / 2^{level + 1})^2$ because the size of the product is half the size of the input at this level, however since we have 4 products, this just simplifies to $(n / 2^{level})^2$.

Now let's bring all together, we have $8^{level}$ nodes per level, for each node we have to perform $(n / 2^{level})^2$ steps to create the result, multipling them together and doing some simplification we get:

$$
8^{level} (n / 2^{level})^2 = 2^{3 level} n^2 / 2^{2level} = 2^{level} n^2
$$

Now given that the complexity is the sum of work for all levels, we get:

$$
\sum_{level = 0}^{log(n)} 2^{level} n^2 = n^2 (1 + 2 + 4 + ... + 2^{log(n)}) = n^2 (1 + 2 + 4 + ... + n) 
$$

Given that the definition of big-O involves dropping all constant factors and lower order terms, hence the amount of work required to do the divide an conquer matrix multiplication is:

$$
O(n^3)
$$

Hence, it is not an improvement!

## Closest Pair Algorithm

Based on [this video series](https://www.youtube.com/watch?v=3pUOv_ocJyA&list=PLEGCF-WLh2RLHqXx6-GZr_w7LgqKDXxN_&index=16)


The closest pair problem involves finding the closest pair between 2 points in the plan.

**Problem**: Given the list of points [p1, p2, ..., pn] where each point has x and y coordinates pi = [xi, yi], find the pair of points that are closest to each other in the plane (where distance between 2 points is the euclidean distance).

**Input**: A list of assorted points [p1, p2, p3, ..., pn] where each point has x and y coordinates pi = [xi, yi]

**Ouput**: A pair of points [p1, p2] that are the closest in the list. The pair of points should be sorted by x coordinate. For example [(1,2), (2, 3)] is valid.


**Example**:

Input: [(0, 10), (1, 1), (10, 10) (1.5, 1.5)]
Output: [(1, 1), (1.5, 1.5)]


**Analysis**:

A trivial solution for this problem uses a brute force approach, where we will iterate over every unique pairs of points, finding the pair with the minimum distance.

```
min_distance = float('inf')
closest_pair = None
for i, p1 in enumerate(input):
    for j in range(i, len(input):
       p2 = input[j]
       if euc_distance(p1, p2) < min_distance:
          min_distance = euc_distance(p1, p2)
          closest_pair = [p1, p2]
          
if not closest_pair:
    return None # failure case
    
return sorted_by_x(closest_pair)
```

The problem with the above solution is that it is $O(n^2)$, since the number of operations in this 2 nested for loops is n choose 2, which is $n(n-1) / 2$, which after dropping the constants and lower order terms becomes $O(n^2)$



So obviously the brute force approach is not great, so **can we do better?** Specifically can we achieve nlog(n) performance similar to merge sort?

Turns out we could with a divide and conquer approach, we could do the following:

- Given an input point list P, we could sort this point list based on the x coordinate and y coordinate, this will form to lists Px and Py sorted by their respective coordinates. This operation is $O(nlog(n))$.
- We could then define a method `closestPair(Px, Py)` which will take the two sorted lists and outputs the closest pair
- Using Px, we could split points into 2 equal groups, we could call them left points and right points:

![closest_pair_algorithm](../diagrams/exported/closest_pair_algorithm.png)

- We could recursively call `closestPair(Px, Py)` over each of the left / right batches of points
- We need to be able to merge the results from the 2 recursive calls
- To do so we to consider the following clever fact, explained in [this video](https://www.youtube.com/watch?v=7tiafUFrlBw&list=PLEGCF-WLh2RLHqXx6-GZr_w7LgqKDXxN_&index=17):


![closest_pair_proof_of_correctness](../diagrams/exported/closest_pair_proof_of_correctness.png)

- Given that $\delta$ is the minimum of the results from the 2 closest pairs recursive calls.
- There are 2 scenarios to consider:
  1. Either the closest pair is fully in the right or left batch
  2. The closest pair split between the right / left batches
- The first thing to observe about the above diagram is that there can be no 2 points in the same box, because if 2 points are in the same box, they will be at most $\sqrt2 \delta/2$ which is obviously less than $\delta$, and since these 2 points have to be either fully on the right or on the left batch, then is not possible because we already established that $\delta$ is the minimum of the closest pair distances in both the right and the left batches. Therefore, there must be one point per box in the above diagram.
- Since there must be only one point per box, and the closest pair must be smaller than $\delta$ in distance, hence we the closest pair must be within the 8 drawn boxes (where each point would occupy a box of the 8). Note that we didn't consider more than 8 boxes since adding anymore boxes will result in pairs that have a distance larger than $\delta$ which is obviously not going to work
- Since we established that the closest pair must be within these 8 boxes, hence if we sort the points by y, the closest pair must be at most 7 indicies apart.


Let's not implement this algorithm:


In [6]:
// Type alias signifying a point
type Point = (f64, f64);


// Merge sort that sorts points based on a given comparison function
fn merge_sort_points(points: &mut [Point], compare: fn(Point, Point)->bool) {
    if points.len() <= 1 {
        return;
    }
    
    let mid_index = points.len() / 2;
    merge_sort_points(&mut points[..mid_index], compare);
    merge_sort_points(&mut points[mid_index..], compare);
    
    let mut i = 0;
    let mut j = mid_index;
    
    let mut output: Vec<Point> = Vec::new();
    
    while i < mid_index || j < points.len() {
        if (j >= points.len() || (i < mid_index && compare(points[i], points[j]))) {
            output.push(points[i]);
            i += 1;
        }
        else {
            output.push(points[j]);
            j += 1;
        }
    }
    
    points.copy_from_slice(&output);
}

// Compare based on x coordinates
fn compare_x(p1: Point, p2: Point) -> bool {
    p1.0 < p2.0
}

// Compare based on y coordinates
fn compare_y(p1: Point, p2: Point) -> bool {
    p1.1 < p2.1
}

In [7]:
// Test merge_sort_points
fn test_merge_sort_points() {
    let mut test_vec = vec![(1.0, 0.0), (0.0, 1.0), (2.0, 10.0)];
    merge_sort_points(&mut test_vec, compare_x);
    assert_eq!(test_vec, vec![(0.0, 1.0), (1.0, 0.0), (2.0, 10.0)]);
    let mut test_vec = vec![(1.0, 0.0), (0.0, 1.0), (2.0, -10.0)];
    merge_sort_points(&mut test_vec, compare_y);
    assert_eq!(test_vec, vec![(2.0, -10.0), (1.0, 0.0), (0.0, 1.0)]);
}

test_merge_sort_points();

In [8]:
use std::cmp;

// Function that calculates the euclidean distance between 2 points
fn euc_distance(p1: &Point, p2: &Point) -> f64 {
    ((p1.0 - p2.0).powi(2) + (p1.1 - p2.1).powi(2)).sqrt()
}

// Test euc distance
assert_eq!(euc_distance(&(0.0, 0.0), &(0.0, 1.0)), 1.0);
assert_eq!(euc_distance(&(0.0, 0.0), &(1.0, 1.0)), (2.0f64).sqrt());

fn ordered_pair(pair: &mut (Point, Point)) {
    if (pair.0.0 > pair.1.0) || (pair.0.0 == pair.1.0 && pair.0.1 > pair.1.1) {
        let tmp = pair.0.clone();
        pair.0 = pair.1;
        pair.1 = tmp;
    }
}
let mut test_pair = ((1.0, 0.0), (0.0, 1.0));
ordered_pair(&mut test_pair);
assert_eq!(test_pair, ((0.0, 1.0), (1.0, 0.0)));

let mut test_pair = ((0.0, 1.0), (0.0, 0.0));
ordered_pair(&mut test_pair);
assert_eq!(test_pair, ((0.0, 0.0), (0.0, 1.0)));

let mut test_pair = ((0.0, 0.0), (0.0, 0.0));
ordered_pair(&mut test_pair);
assert_eq!(test_pair, ((0.0, 0.0), (0.0, 0.0)));

In [9]:
// Function that calculates the closest pair in a given list of points recursively
// points_x: The list of points sorted by x-coordinate
// points_y: The list of points sorted by y-coordinate
fn recurse_closest_pair(points_x: &[Point], points_y: &[Point]) -> Result<((Point, Point), f64), String> { // Returns the pair along with the distance
    if points_x.len() != points_y.len() {
        return Err("Points x and y are not the same length".to_string());   
    }

    if points_x.len() <= 1 {
        return Err("Points are of length 0 or 1".to_string());   
    }
    
    // Base Case, 2 points
    if points_x.len() == 2 {
        return Ok(((points_x[0], points_x[1]), euc_distance(&points_x[0], &points_x[1])));
    }

    // Base case, 3 points
    if points_x.len() == 3 {
        let d1 = euc_distance(&points_x[0], &points_x[1]);
        let d2 = euc_distance(&points_x[0], &points_x[2]);
        let d3 = euc_distance(&points_x[1], &points_x[2]);
        
        let mut min_d = d1;
        let mut min_pair = (points_x[0], points_x[1]);
        
        if d2 < min_d
        {
            min_d = d2;
            min_pair = (points_x[0], points_x[2]);
        }
        if d3 < min_d
        {
            min_d = d3;
            min_pair = (points_x[1], points_x[2]);
        }
        
        return Ok((min_pair, min_d));
    }
    // Length of points from now on is at least 4
    
    let mut mid_index = points_x.len() / 2;
    let mid_x = points_x[mid_index].0;
    
    
    // We want all points that are to the left of the mid_index to have strictly lower x-coordinates.
    // That's because the assumption is that all points that have an x-coordinate < mid_x should be 
    // in the left batch, and the left batch is all the points in [0, mid_index)
    // So this while loop will keep decrementing the the mid_index, till we have strictly lower x value, or the
    // left batch has 2 points (which is the minimum base case for recurse_closest_pair).
    while mid_index > 2 && points_x[mid_index - 1].0 == mid_x {
        mid_index -= 1;
    }
       
    let mut points_y_left: Vec<Point> = Vec::new();
    let mut points_y_right: Vec<Point> = Vec::new();
    
    for p in points_y {
        // The first 2 points of points_x will always be in the left batch (this is because we have the mid_index > 2)
        // in the while loop above, which ensures we have at least 2 points in the left batch even if they have the
        // same x-coordinate as the middle point (which lies in the right batch).
        if *p == points_x[0] {
            points_y_left.push(*p);
        }
        else if *p == points_x[1] {
            points_y_left.push(*p);
        }
        // The mid_point lies in the right batch
        // So any point that has an x-coordinate that is strictly smaller than the mid_point
        // will be in the left batch (except the first 2 points as explained above).
        else if p.0 < mid_x {
            points_y_left.push(*p);
        }
        else {
            points_y_right.push(*p);
        }
    }
    
    // Recursively calculate the left and right closest points
    let (pair_left, d_left) = recurse_closest_pair(&points_x[..mid_index], &points_y_left)?;
    let (pair_right, d_right) = recurse_closest_pair(&points_x[mid_index..], &points_y_right)?;
    
    
    // Get the minimum distance between pairs right and left
    let mut delta;
    let mut closest_pair;
    if d_left < d_right {
        delta = d_left;
        closest_pair = pair_left;
    }
    else {
        delta = d_right;
        closest_pair = pair_right;
    }
    
    // Populate Sy, which contains all the points that are delta distance right and left from the boundary at mid_x.
    let mut Sy = Vec::new();
    for p in points_y {
        if (p.0 - mid_x).abs() <= delta {
            Sy.push(p);
        }
    }
    
    // Use the clever trick that closest split pair must be max 7 positions away
    if Sy.len() >= 2 {
        for i in 0..(Sy.len() - 1) {
            for j in i+1..(Sy.len().min(i+7)) {
                let d = euc_distance(Sy[i], Sy[j]);
                if d < delta {
                    delta = d;
                    closest_pair = (*Sy[i], *Sy[j]);
                }
            }
        }
    }
    

    return Ok((closest_pair, delta));
}


// The final closest pair function
// points: The list of points.
fn closest_pair(points: &Vec<Point>) -> Result<((Point, Point), f64), String> {
    let mut points_x = points.clone();
    let mut points_y = points.clone();

    merge_sort_points(&mut points_x, compare_x);
    merge_sort_points(&mut points_y, compare_y);

    let (mut closest_pair, d) = recurse_closest_pair(&points_x, &points_y)?;
    ordered_pair(&mut closest_pair);
    return Ok((closest_pair, d));
    
}


let points = vec![(1.0, 10.0), (2.0, 3.0), (1.5, 5.0), (1.5, 11.0)];

let (c_pair, d) = closest_pair(&points).unwrap();
println!("closest pair: {:?}, distance: {:?}", c_pair, d);

closest pair: ((1.0, 10.0), (1.5, 11.0)), distance: 1.118033988749895


In [10]:
fn closest_pair_brute_force(points: &Vec<Point>) -> Result<((Point, Point), f64), String> {
    let mut min_d = f64::INFINITY;
    let mut min_pair: Option<(Point, Point)> = None;
    
    for (i, p1) in points.iter().enumerate() {
        for j in i+1..points.len() {
            let p2 = points[j];
            let d = euc_distance(&p1, &p2);
            if d < min_d {
                min_d = d;
                min_pair = Some((*p1, p2));
            }
        }
    }
    let mut closest_pair = min_pair.ok_or("Min Pair not found".to_string())?;
    ordered_pair(&mut closest_pair);
    return Ok((closest_pair, min_d));
}

let (c_pair, d) = closest_pair_brute_force(&points).unwrap();
println!("closest pair: {:?}, distance: {:?}", c_pair, d);

closest pair: ((1.0, 10.0), (1.5, 11.0)), distance: 1.118033988749895


In [11]:
// Test the closest_pair routine

fn test_closest_pair() {
    // Random vector.
    let points = vec![(1.0, 10.0), (2.0, 3.0), (1.5, 5.0), (1.5, 11.0)];
    assert_eq!(closest_pair(&points), closest_pair_brute_force(&points));
 
    // One point should fail
    let points = vec![(1.0, 10.0)];
    assert!(closest_pair(&points).is_err());
    assert!(closest_pair_brute_force(&points).is_err());
    
    // Two points should succeed, points should be ordered
    let points = vec![(5.0, 0.0), (1.0, 10.0)];
    assert_eq!(closest_pair(&points).unwrap(), (((1.0, 10.0), (5.0, 0.0)), euc_distance(&points[0], &points[1])));
    
    // Same x coordinate all along should succeed.
    let points = vec![(0.0, 10.0), (0.0, 0.01), (0.0, 0.0), (0.0, 31.0), (0.0, 70.0)];
    assert_eq!(closest_pair(&points), closest_pair_brute_force(&points));
    assert_eq!(closest_pair(&points).unwrap(), (((0.0, 0.0), (0.0, 0.01)), 0.01));
    
    // Same y coorinates
    let points = vec![(10.0, 0.0), (0.01, 0.0), (0.0, 0.0), (31.0, 0.0), (70.0, 0.0)];
    assert_eq!(closest_pair(&points), closest_pair_brute_force(&points));
    assert_eq!(closest_pair(&points).unwrap(), (((0.0, 0.0), (0.01, 0.0)), 0.01));
    
    
}

test_closest_pair();

In [12]:
// Let's now stress test our closest_pair function using random inputs.

:dep rand


// Random point generation for testing 
use rand::Rng;
use std::panic;


// Function that will generate random points for testind
fn generate_points(n: usize, x_range: (f64, f64), y_range: (f64, f64)) -> Vec<Point> {
    let mut rng = rand::thread_rng();
    let (xmin, xmax) = x_range;
    let (ymin, ymax) = y_range;

    (0..n)
        .map(|_| {
            let x = rng.gen_range(xmin..=xmax);
            let y = rng.gen_range(ymin..=ymax);
            (x, y)
        })
        .collect()
}

// Test with random inputs
for i in 0..10000 {
    let points = generate_points(1000, (-100.0, 100.0), (-100.0, 100.0));
    assert_eq!(closest_pair(&points), closest_pair_brute_force(&points));
    if i % 1000 == 0 {
        println!("Iteration {:?}", i);
    }
}

Iteration 0
Iteration 1000
Iteration 2000
Iteration 3000
Iteration 4000
Iteration 5000
Iteration 6000
Iteration 7000
Iteration 8000
Iteration 9000


()

In [14]:
// Let's now compare the performance of the 2 methods with very large inputs
use std::time::Instant;

let lots_of_points = generate_points(100000, (-100.0, 100.0), (-100.0, 100.0));

let start = Instant::now();
closest_pair(&lots_of_points);
let duration = start.elapsed();
println!("Divide and conquer approach: {:?}", duration);

let start = Instant::now();
closest_pair_brute_force(&lots_of_points);
let duration = start.elapsed();
println!("Brute force approach: {:?}", duration);

Divide and conquer approach: 80.981462ms
Brute force approach: 8.098401544s


### Analysis of the closest pair algorithm

As you can see, the divide and conquer approach is significantly more performant compared to the brute force approach. (see above results where the divide and conquer approach is a 100 times faster than the brute force approach).

The `clooset_pair` runs at $O(nlog(n))$ since we only perform linear work outside of the 2 recursive calls. Note that we have a double for loop when we are looking for the closest split pair in Sy, but since the inner loop is limited to 7 iterations, we effectievly perform linear work.

## The Master Method

This section outlines the master method.

> The master method is a formal way to analyze recursive divide and conquer algorithms.

Checkout the video series for the master method [here](https://www.youtube.com/watch?v=6dGDcszz2DM&list=PLEGCF-WLh2RLHqXx6-GZr_w7LgqKDXxN_&index=18).

The master method has the following assumptions:
1. Everytime we divide into smaller subproblem, all subproblems have the same size.
2. We perform $O(n^d)$ work merging the results after recursion.

You could then write the recursion as:

$$
T(n) \leq aT(\frac{n}{b}) + n^d
$$

Where:
- $a$: Is the branching factor, which is the number of subproblems we divide as we step down the recursion tree.
- $b$: The input shrinkage factor for every subproblem
- $d$: The amount of polynomial work we do outside of each subproblem.

There are three different scenarios that the master method presents:
1. if $a == b^d$, then the complexity is $O(n^dlog(n))$ - we perform the same amount of work per level ($n^d$) and we have $log(n)$ levels.
2. If $a < b^d$, then the complexity is $O(n^d)$ - The work is dominated by the root node of the recursion tree, which is n^d
3. if $a > b^d$, then the complexity is $O(n^{log_ba})$ - The work is dominated by the leaf nodes of the recursion tree, we have $a^{log_bn}$ leaf nodes which is equivelant to $n^{log_ba}$ (take log of each side to prove it).



## The Bubble sort algorithm